In [158]:
import lmdb
import torch
from ocpmodels.datasets import SinglePointLmdbDataset
import pandas as pd
import pickle

In [151]:
dataset_origin_path = "/Users/korovin/Bitrix24/AI for MS Group Folder/catalyst/structures.pkl"
dataset_origin_old_path = "../../ocp_datasets/data/is2re/10k/train/data.lmdb"
dataset_target_path = "../../ocp_datasets/data/is2re/10k/train/data_mod.lmdb"

In [152]:
dataset_origin_old = SinglePointLmdbDataset({"src": dataset_origin_old_path})

In [153]:
dataset_origin_old[0]['edge_index']

tensor([[69, 83, 75,  ..., 63, 48, 42],
        [ 0,  0,  0,  ..., 85, 85, 85]])

In [154]:
dataset_origin = pd.read_pickle(dataset_origin_path)

In [155]:
dataset_origin[0]['edge_index_new']

array([[ 0, 17,  0, ..., 80, 85, 26],
       [17,  0, 39, ..., 85, 26, 85]])

In [160]:
dataset_target = lmdb.open(
    dataset_target_path,
    map_size=1099511627776 * 2,
    subdir=False,
    meminit=False,
    map_async=True,
)

idx = 0

for ii, data_object_origin_old in enumerate(dataset_origin_old):
        
        # Substitute: edge_index -> edge_index_new
        data_object = dataset_origin_old[ii]
        edge_index = torch.from_numpy(dataset_origin[ii]['edge_index_new'])
        data_object['edge_index'] = edge_index
        
        # Write to LMDB
        txn = dataset_target.begin(write=True)
        txn.put(f"{idx}".encode("ascii"), pickle.dumps(data_object, protocol=-1))
        txn.commit()
        dataset_target.sync()
        if idx % 1000 == 0:
            print(idx)
        idx += 1

dataset_target.close()
print("done")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [161]:
dataset_target = SinglePointLmdbDataset({"src": dataset_target_path})

In [162]:
dataset_target[0]['edge_index']

tensor([[ 0, 17,  0,  ..., 80, 85, 26],
        [17,  0, 39,  ..., 85, 26, 85]])